In [ ]:
%matplotlib widget
%load_ext autoreload

In [ ]:
%autoreload 2

from fimpylab import LightsheetExperiment
from pathlib import Path
import numpy as np
from matplotlib import  pyplot as plt
import flammkuchen as fl
import pandas as pd

from lotr.utils import zscore
from lotr.behavior import get_bouts_props_array
from lotr.default_vals import TURN_BIAS, GCAMP_TAU

In [ ]:
# path = Path("/Users/luigipetrucco/Desktop/source_data_h2b/210803_f3cwccw")
path = Path("/Users/luigipetrucco/Desktop/source_data/210716_f6_clol")
exp = LightsheetExperiment(path)

In [ ]:
beh_df = exp.behavior_log

In [ ]:
traces = fl.load(path / "filtered_traces.h5", "/detr")
traces_raw = fl.load(path / "data_from_suite2p_unfiltered.h5", "/traces").T

df = fl.load(path / "bouts_df.h5")
df["idx_imaging"] = np.round(df["t_start"] * 5).astype(np.int)
n_pts = traces.shape[0]

In [ ]:
master_path = Path("/Users/luigipetrucco/Desktop/source_data_batch2")

In [ ]:
for path in master_path.glob("*_f*"):
    reg_df = fl.load(path / "motor_regressors.h5")
    
    plt.figure()
    plt.title(path.name)
    plt.scatter(reg_df["all_bias_abs"], reg_df["all_bias_abs_dfdt"])
    plt.xlim(-1, 1)
    plt.ylim(-0.3, 0.6)
    

In [ ]:
reg_dict = create_motor_regressors(n_pts, df, fn=5, min_bias=0.05)

In [ ]:
plt.figure(figsize=(9, 4))
for i, (name, reg) in enumerate(reg_dict.items()):
    plt.plot(reg + i*2)
    plt.text(0, i*2+0.01, name, fontsize=8)

In [ ]:
from lotr.utils import pearson_regressors

In [ ]:
traces_derivative = np.zeros(traces.shape)
traces_derivative[:-1, :] = np.diff(traces, axis=0)

reg_mat = pearson_regressors(traces, reg_dict.values).T
reg_mat_diff = pearson_regressors(traces_derivative, reg_dict.values).T

In [ ]:
np.concatenate([reg_mat, reg_mat_diff], axis=0).shape

In [ ]:
for

In [ ]:
plt.figure()
r_i = 3
plt.plot(zscore(traces_raw[:, np.argsort(-reg_mat[r_i, :])[0]]))
plt.plot(zscore(traces[:, np.argsort(-reg_mat[r_i, :])[0]]))
plt.plot(zscore(reg_dict.values[:, r_i]))

In [ ]:
get_bouts_props_array(n_pts, df,
                           selection=d, value=v, min_bias=min_bias)

In [ ]:
get_bouts_props_array(n_pts, bouts_df, min_bias=0.05,
                           selection="all", value="bias")
get_bouts_props_array(n_pts, bouts_df, min_bias=0.05,
                           selection="all", value="bias")
get_bouts_props_array(n_pts, bouts_df, min_bias=0.05,
                           selection="all", value="bias")

In [ ]:
plt.figure()
plt.plot(beh_df["t"], beh_df["tail_sum"])

In [ ]:

df.columns

In [ ]:

theta_turned = get_theta_turned_array(traces.shape[0], df, min_bias=0)

fwd = theta_turned.copy()
fwd[np.abs(fwd) > TURN_BIAS] = 0
fwd[np.abs(fwd) > 0] = 1 

abs_motion = theta_turned.copy()
abs_motion[np.abs(abs_motion) > 0] = 1

lf = theta_turned.copy()
lf[lf > -TURN_BIAS] = 0

rt = theta_turned.copy()
rt[rt < TURN_BIAS] = 0

regr_dict = dict(fwd=fwd,
                 left=lf,
                 right=rt,
                 all_motion=)

In [ ]:
plt.figure()
plt.plot(fwd)
plt.plot(lf)
plt.plot(rt)

In [ ]:



from fimpylab import autoload_experiment

import pandas as pd

import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()

plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Libertinus Sans']

# %autoreload
from lotr.utils import zscore
from lotr.data_loading import preprocess_traces
from lotr.pca import pca_and_phase, get_fictive_trajectory, fictive_trajectory_and_fit

In [ ]:
plt.close("all")



In [ ]:
df = df[(df["t_start"] > 10) & (df["t_start"] < 2020)]
df["idx_imaging"] = np.round(df["t_start"]).astype(np.int) * 5

sel = (df["t_start"] > 10) & (df["t_start"] < 2020)
bouts_traces[df.loc[sel, "idx_imaging"]] = df.loc[sel, "bias"]
# bouts_traces[idx_r] = -1
plt.figure(figsize=(5, 2))
plt.hist(df["bias"], 10, density=True, zorder=-100)
plt.scatter(df["bias"], df["peak_vig"], 100, c=cols[1])

print(len(df), len(idx_l), len(idx_r))

start = 0 #4800
end = 7000
traces = traces[start:end, :]
idx_l = (df.loc[(df["bias"] > 0.2) & (df["t_start"] > 10), 
                "t_start"].values*5 - start).astype(np.int)
idx_r = (df.loc[(df["bias"] < -0.2) & (df["t_start"] > 10), 
                "t_start"].values*5 - start).astype(np.int)
idx_l = idx_l[(idx_l > 0) & (idx_l < traces.shape[0])]
idx_r = idx_r[(idx_r > 0) & (idx_r < traces.shape[0])]
bouts_traces = np.zeros(traces.shape[0])
bouts_traces[idx_l] = 1
bouts_traces[idx_r] = -1

In [ ]:
t_lims = (1000, 5100)
t_slice = slice(*t_lims)
min_bias = 0.05
fn = 5
tau_s = 5


diff_traces = np.abs(np.diff(traces, axis=0))

n_cells = traces.shape[1]
cc_motor = np.zeros(n_cells)
cc_motor_integr = np.zeros(n_cells)

for i in tqdm(range(n_cells)):
    cc_motor_integr[i] = (np.corrcoef(diff_traces[:, i], all_motions_reg)[0, 1])
    cc_motor[i] = (np.corrcoef(traces[1:, i], all_motions_reg)[0, 1])#

In [ ]:
corr_mat = np.corrcoef(traces[t_slice, :].T)
n_pairs = 5
selected = []
incl_cc = []
for n in range(n_pairs):
    pair = np.unravel_index(np.nanargmin(corr_mat), corr_mat.shape)
    selected.extend(pair)
    incl_cc.append(corr_mat[pair[0], pair[1]])
    for p in pair:
        corr_mat[p, :] = np.nan
        corr_mat[:, p] = np.nan
np.array((selected))

corr_mat = np.corrcoef(traces[:, :].T)

f = plt.figure(figsize=(3, 3))
plt.scatter(cc_motor, cc_motor_integr, s=10, label="excluded")
#plt.scatter(cc_motor[selected], cc_motor_integr[selected])
x = np.arange(-0.2, cc_motor.max(), 0.05)

c = 0.3
o = 0.05
plt.plot(x, x*c +o, c=(0.3,)*3)
selected = np.argwhere(np.min(corr_mat, 0) < -0.6)[:, 0]
# selected = fl.load(path / "selected_cc.h5")
# selected = np.argwhere((cc_motor_integr > 0.1) & (np.abs(cc_motor) < 0.3))[:, 0]
#selected = np.argwhere((cc_motor_integr > cc_motor*c + o) & (np.abs(cc_motor) < 0.4))[:, 0]
# selected = np.argwhere((cc_motor_integr > cc_motor*c + o))[:, 0]
# selected = np.argwhere((np.abs(cc_motor) < 0.1))[:, 0]
# selected = np.argwhere((np.min(corr_mat, 0) < -0.5)  & (np.abs(cc_motor) < 0.4))[:, 0]
selected = np.argwhere(((cc_motor_integr > cc_motor*c + o) & (np.abs(cc_motor) < 0.8) & \
                        (np.abs(cc_motor_integr)> 0.05)) | \
                      (np.min(corr_mat, 0) < -0.9))[:, 0]
#selected = [ 344,  518,  611,  775,  780,  905,  971,  976,  979,  980,  986,
#       1067, 1078, 1100, 1112, 1188, 1234]
#selected = np.concatenate([selected, np.argmin(corr_mat[selected, :], axis=1)])
plt.ylim(-0.3, 0.8)
plt.xlim(-0.6, 1)
#selected = selected[(np.abs(cc_motor[selected]) < 0.2)]
plt.scatter(cc_motor[selected], cc_motor_integr[selected], s=10, label="included")
# pcaed, phase = pca_and_phase(traces[2000:8000, selected], traces[:, selected])
plt.legend(frameon=False)
plt.xlabel("cc. traces - motor regressor")
plt.ylabel("cc. d(traces)/dt - regressor")
sns.despine()
# f.savefig("/Users/luigipetrucco/Desktop/regbased_incl.pdf")

In [ ]:
pcaed_t, phase_t = pca_and_phase(traces[t_slice, selected].T, traces[t_slice, selected].T)

plt.figure(figsize=(7, 3))
thr = 35
sel = (pcaed_t[:, 0]**2+pcaed_t[:, 1]**2)**(1/2) > thr
plt.scatter(pcaed_t[:, 0], pcaed_t[:, 1], c=sel)
plt.axis("equal")

#selected = selected[sel]
# pcaed, phase = pca_and_phase(traces[t_slice, selected], traces[:, selected])
pcaed, phase = pca_and_phase(traces[t_slice, selected], traces[:, selected])

In [ ]:
plt.figure(figsize=(7, 2.5))
plt.plot(traces[:, selected] + 4)
plt.plot(mot_reg_base, c="k")
print(len(selected))
plt.show()

In [ ]:
# plt.close("all")
plt.figure(figsize=(3., 3.))
pcaed_crop = pcaed[:, :]
phase = np.angle((pcaed[:, 0]) + 1j * (pcaed[:, 1]))
plt.plot(pcaed_crop[t_slice, 0], pcaed_crop[t_slice, 1], 
         c=(0.6,)*3, lw=0.5, zorder=-100) # , c=phase, cmap="twilight", lw=3)
cb = plt.scatter(pcaed_crop[t_slice, 0], pcaed_crop[t_slice, 1], 
                 c=phase[t_slice], lw=0.5, s=5, cmap="twilight",) 
# cbar = add_cbar((0.9, 0.83, 0.02, 0.12), cb, label="phase", ticks=[-np.pi/1.1, np.pi/1.1], 
#         ticklabels=["-π", "π"])
sns.despine()

In [ ]:
unwrapped_phase = np.unwrap(phase)
traj, params = fictive_trajectory_and_fit(unwrapped_phase, df, min_bias=0.)
print(params)

In [ ]:
np.corrcoef(traj, unwrapped_phase)

In [ ]:
plt.figure(figsize=(7, 3))
plt.scatter(np.arange(len(traj)), unwrapped_phase*params[1] + params[0], c=phase, cmap="twilight", s=1)
plt.plot(traj, c=cols[1])

In [ ]:
from sklearn.decomposition import PCA
from circle_fit import hyper_fit
comp0, comp1 = 0, 1

traces_fit = traces[2000:8000, selected].T
traces_transform = traces_fit
if traces_transform is None:
    traces_transform = traces_fit

# Compute PCA and transform traces:
pca = PCA(n_components=5).fit(traces_fit)
pcaed_t = pca.transform(traces_transform)

# Fit circle:
hf_c = hyper_fit(pcaed[:, [comp0, comp1]])

# Compute phase, after subtracting center of the circle
phase_t = np.angle((pcaed_t[:, 0] - hf_c[0]) + 1j * (pcaed_t[:, 1] - hf_c[1]))

plt.figure(figsize=(7, 3))
plt.scatter(pcaed_t[:, 0], pcaed_t[:, 1], c=phase_t, cmap="twilight")
plt.axis("equal")

In [ ]:
# plt.close("all")
plot_t_s = 10
plot_t_pts = int(plot_t_s * fn)

rot_wnd_s = 1
rot_wnd_pts = int(rot_wnd_s * fn)

f, axs = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
# [ax.set_xlabel("PC 1") for ax in axs]
# axs[0].set_ylabel("PC 2")

cbars = []
for x, ax, idx_list, cmap, title in zip(np.arange(3), 
                                     axs, 
                                     [idx_l, idx_r, random_trig], 
                                     ["Blues", "Reds", "gray_r"],
                                     ["Left bouts", "Right bouts", "Shuffle"]):   
    for l in idx_list[:]:
        try:
            crop_seg = pcaed[l:l+plot_t_pts, :2]
            crop_seg = crop_seg - crop_seg[0, :]
            th = np.arctan2(crop_seg[rot_wnd_pts, 0] - crop_seg[0, 0], 
                            crop_seg[rot_wnd_pts, 1] - crop_seg[0, 1])

            rot_mat = np.array([[np.cos(th), np.sin(th)], [-np.sin(th), np.cos(th)]]).T 
            crop_seg = (rot_mat @ crop_seg[:, :].T).T
            ax.plot(crop_seg[::3, 0], crop_seg[::3, 1], c=(0.4,)*3, lw=0.3, zorder=-100)
            ax.scatter(crop_seg[::3, 0], crop_seg[::3, 1], c=np.arange(len(crop_seg[::3, 1])), cmap=cmap, s=1)
            #cp = color_plot(crop_seg[::3, 0], crop_seg[::3, 1], ax=ax,
             #          cmap=cmap, lw=1)
        except IndexError:
            pass
    cbars.append(add_cbar((0.93, 0.8 + 0.026*x, 0.06, 0.023), cp, label="time (s)" if x==2 else "", 
                          ticks=[], orientation="horizontal"))
    ax.set_title(title)
    # elif x == 1:
    # elif x == 2:
    ax.axvline(0, lw=0.5, c=(0.4,)*3)
    ax.axhline(0, lw=0.5, c=(0.4,)*3)

plt.tight_layout()
sns.despine()

cbars[0].set_ticks([0, 0.95])
cbars[0].set_ticklabels([0, plot_t_s])

In [ ]:
plt.close("all")
plot_t_s = 8
plot_t_pts = int(plot_t_s * fn)

f, axs = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
[ax.set_xlabel("PC 1") for ax in axs]
axs[0].set_ylabel("PC 2")

cbars = []
for x, ax, idx_list, cmap, title in zip(np.arange(3), 
                                     axs, 
                                     [idx_l, idx_r, random_trig], 
                                     ["Blues", "Reds", "gray_r"],
                                     ["Left bouts", "Right bouts", "Shuffle"]):   
    for l in idx_list:
        try:
            crop_seg = pcaed[l:l+plot_t_pts, :2]
            # crop_seg = crop_seg - crop_seg[0, :]
            #cp = color_plot(crop_seg[::3, 0], crop_seg[::3, 1], ax=ax,
            #           cmap=cmap, lw=1)
            ax.plot(crop_seg[::3, 0], crop_seg[::3, 1], c=(0.4,)*3, lw=0.3, zorder=-100)
            ax.scatter(crop_seg[::3, 0], crop_seg[::3, 1], c=np.arange(len(crop_seg[::3, 1])), cmap=cmap, s=1)
            
        except (IndexError, ValueError):
            pass
    cbars.append(add_cbar((0.92, 0.8 + 0.026*x, 0.06, 0.023), cp, label="time (s)" if x==2 else "", 
                          ticks=[], orientation="horizontal"))
    ax.set_title(title)
    # elif x == 1:
    # elif x == 2:
        
plt.tight_layout()
sns.despine()

cbars[0].set_ticks([0, 0.8])
cbars[0].set_ticklabels([0, plot_t_s])

In [ ]:
path

In [ ]:
fl.save(path / "selected_new.h5", selected)

In [ ]:
master_path = Path("/Volumes/Shared/experiments/E0040_motions_cardinal_old/v15_playback")
all_list = list(master_path.glob("[0-9]*_f[0-9]*"))
all_valid = list(master_path.glob("[0-9]*_f[0-9]*/selected.h5"))
print(f"{len(all_valid)}/{len(all_list)}")

In [ ]:
plt.close("all")
plot_t_s = 8
plot_t_pts = int(plot_t_s * fn)

f, axs = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
[ax.set_xlabel("PC 1") for ax in axs]
axs[0].set_ylabel("PC 2")

cbars = []
for x, ax, idx_list, cmap, title in zip(np.arange(3), 
                                     axs, 
                                     [idx_l, idx_r, random_trig], 
                                     ["Blues", "Reds", "gray_r"],
                                     ["Left bouts", "Right bouts", "Shuffle"]):   
    for l in idx_list:
        try:
            crop_seg = pcaed[l:l+plot_t_pts, :2]
            # crop_seg = crop_seg - crop_seg[0, :]
            cp = color_plot(crop_seg[:, 0], crop_seg[:, 1], ax=ax,
                       cmap=cmap, lw=1)
        except IndexError:
            pass
    cbars.append(add_cbar((0.92, 0.8 + 0.026*x, 0.06, 0.023), cp, label="time (s)" if x==2 else "", 
                          ticks=[], orientation="horizontal"))
    ax.set_title(title)
    # elif x == 1:
    # elif x == 2:
        
plt.tight_layout()
sns.despine()

cbars[0].set_ticks([0, 0.8])
cbars[0].set_ticklabels([0, plot_t_s])